# Optimisation via Simulation Tutorial
## S
By Christine S.M Currie and Tom Monks

In [1]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [2]:
from ovs.toy_models import (custom_guassian_model,
                            guassian_sequence_model,
                            random_guassian_model,
                            ManualOptimiser)

This tutorial will be making use of the following models.  

### Creating a simple sequence of normal distributions

A simple model can be created using the `guassian_sequance_model()` function.  This creates a simulation model where the output of each design follows a $N -(\mu_{i}, 1.0)$ :

The function accepts three keyword arguments:

* **start** - int, the first mean in the sequence (inclusive)
* **end** - int, the last mean int the sequence (inclusive)
* **step** - int, the difference between mean i and mean i + 1.

For example, the following code creates a simulation model with 10 designs with means 1 to 10 and unit variance.

```python
model = guassian_sequence_model(1, 10)
```

To create a simulation model with 5 designs where $\mu_{i+1} - \mu_i = 2 $ :

```python
model = guassian_sequence_model(1, 10, step=2)
```

### Creating a model with 'unknown' designs

To create a model with a set of unknown designs (within a specified mean and variance tolerance) use 

```python
model = random_guassian_model(mean_low, mean_high, var_low, var_high, n_designs)
```

Where:
* **mean_low** - float, a lower bound on the means of the output distributions
* **mean_high** - float, an upper bound on the means
* **var_low** - float, a lower bound on the variance of the output distributions
* **var_high** - float, an upper bound on the variances.
* **n_designs** - int, the number of designs to create.

### A manual optimisation framework

Before using the **Optimisation via Simulation** procedures, it is recommended that you get a feel for the framework in which the OvS procedures operate.  To do this we will create some models and explore them using a `ManualOptimiser`.  This allows the user to run independent and multiple replications of the model themselves.  The `ManualOptimiser` keeps track of the means, variances and number of replications run for each design.

A `ManualOptimiser` object requires two parameters when it is created.  

* model - object, e.g. a model that is a sequence of normal distributions
* n_designs - the number of designs to be considered.

In [3]:
manual_opt = ManualOptimiser(model=guassian_sequence_model(1, 10),
                             n_designs=10)

In [4]:
print(manual_opt)

ManualOptimiser(model=BanditCasino(), n_designs=10, verbose=False)


Follow Law and Kelton's advise and run 3 initial replications of each design.

In [5]:
manual_opt.simulate_designs(replications=3)

Let's have a look at the replication allocation between and results of each design.

In [6]:
manual_opt.allocations

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [7]:
np.set_printoptions(precision=1) # this is a hack to view at 1 decimal place.
manual_opt.means

array([2.4, 1.8, 2.2, 4.4, 4.5, 6.6, 6.4, 8. , 8.3, 9.4])

In [8]:
manual_opt.sigmas

array([0.1, 0.5, 2.4, 2.1, 4.5, 0.2, 2. , 3.2, 0.4, 2.5])

Now let's run 1 additional replication of the top 3 designs. 
Note - in python arrays are **zero indexed**.  This means that design 1 has index 0.

In [9]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=1)

In [10]:
manual_opt.allocations

array([3, 3, 3, 3, 3, 3, 3, 4, 4, 4], dtype=int32)

In [11]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=2)

In [12]:
manual_opt.means

array([2.4, 1.8, 2.2, 4.4, 4.5, 6.6, 6.4, 7.8, 8.6, 9.6])

In [13]:
manual_opt.sigmas

array([0.1, 0.5, 2.4, 2.1, 4.5, 0.2, 2. , 1.7, 1.3, 1.2])

In [14]:
rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                              var_low=0.5, var_high=2.5,
                              n_designs=10)

manual_opt = ManualOptimiser(model=rnd_model, n_designs=10)

In [15]:
manual_opt.simulate_designs(replications=3)

In [16]:
manual_opt.means

array([20.8,  7.6,  8.8, 18.1, 17.4, 16.3, 12. , 24.1, 14.8, 15.6])

In [17]:
manual_opt.simulate_designs(design_indexes=[0, 3, 4, 6],replications=3)
manual_opt.means

array([19.4,  7.6,  8.8, 17.9, 17.1, 16.3, 11.8, 24.1, 14.8, 15.6])

In [18]:
manual_opt.sigmas

array([9.8, 0.3, 1.4, 1.3, 4.6, 3.3, 1.1, 6.6, 6. , 0.5])

## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [ ]:
from ovs.indifference_zone import KN

In [ ]:
guass_model = guassian_sequence_model(1, 10)

kn = KN(model=guass_model, 
        n_designs=10, 
        delta=0.05, 
        alpha=0.1, 
        n_0=5)

In [ ]:
print(kn)

In [ ]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

In [ ]:
means = [i for i in range(101)]
sigmas = [j*0.1 for j in range(101)]

guass_model = custom_guassian_model(means, sigmas)

kn = KN(model=guass_model, 
        n_designs=100, 
        delta=0.05, 
        alpha=0.1, 
        n_0=5)

In [ ]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [ ]:
from ovs.fixed_budget import OCBA, OCBAM

In [ ]:
guass_model = guassian_sequence_model(1, 10)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=500, 
            delta=5, 
            n_0=5, 
            obj='min')

In [ ]:
print(ocba)

call the `solve()` method to run the optimisation

In [ ]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

## Optimal Computing Budget Allocation Top M (OCBA-m)

OCBA-m extended OCBA to identify the top m designs.  

An object of type `OCBAM` takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **m** - int, $m$ the number of top designs to return
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocbam = OCBAM(model=environment, 
            n_designs=len(designs), 
            budget=500, 
            delta=5, 
            n_0=5, 
            m=3,
            obj='max')

In [ ]:
print(ocbam)

In [ ]:
results = ocbam.solve()
print('best designs:\t{}'.format(np.sort(results)))
print('allocations:\t{}'.format(ocbam._allocations))
print('total reps:\t{}'.format(ocbam._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocbam._means))
print('vars:\t\t{0}'.format(ocbam._vars))
print('SEs:\t\t{0}'.format(ocbam._ses))

In [ ]:
np.argsort(ocba._means)[-3:]

# Evaluation of the methods

In [ ]:
from ovs.evaluation import Experiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

In [ ]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

## Try different budgets

In [ ]:
from ovs.evaluation import GridExperiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[1, 5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=1000)

In [ ]:
results = exp.fit()

In [ ]:
results